Hoja de trabajo 2

Realice una simulación de una sucursal de una cadena de supermercados, utilizando la librería Simpy de Python o utilizando el método de eventos discretos.

En una sucursal de una cadena de supermercados, llegan los clientes a un sistema con M cajas y con un proceso de Poisson a razón de 
 (>0) clientes por hora en promedio. Al llegar un cliente, se forma en una fila en una caja (selecciona la caja que tenga menos personas o en caso que el menor número de personas haya más de una caja, selecciona cualquiera entre esas cajas).  Cada cajero despacha al cliente en un tiempo que tiene una distribución exponencial con parámetro 
 (>0) clientes por hora.

Calcular: 

1. El tiempo promedio de un cliente en la cola.

2. Número de clientes en la cola en promedio.

3. Grado o factor de utilización de cada cajero.



Cajas (Servidores): Representan las cajas de la sucursal y servirán a los clientes.

Clientes: Llegan al sistema y se colocan en la caja con menos personas en fila.

Proceso de llegada de clientes: Los clientes llegan siguiendo un proceso de Poisson.

In [4]:
import simpy
import random

# Parámetros
LAMBDA = 10  # Tasa promedio de llegada de clientes (clientes por hora)
LAMBDA1 = 6 
M = 2       # Número de cajas
SIM_TIME = 480  # Tiempo de simulación (minutos)

# Estadísticas
total_wait_time = 0
total_clients = 0
utilization = 0


def client(env, name, server):
    global total_wait_time
    global total_clients
    global utilization

    arrival_time = env.now

    print(f"{env.now:.2f} - Cliente {name} llega al supermercado")

    with server.request() as req:
        yield req

        wait_time = env.now - arrival_time
        total_wait_time += wait_time

        print(f"{env.now:.2f} - Cliente {name} comienza a ser atendido en la caja {server.idx}. Tiempo de espera: {wait_time:.2f} minutos")

        service_time = random.expovariate(LAMBDA1)
        utilization += service_time
        yield env.timeout(service_time)

        print(f"{env.now:.2f} - Cliente {name} deja la caja {server.idx}")

        total_clients += 1


def run_simulation():
    env = simpy.Environment()
    servers = [simpy.Resource(env, capacity=1) for _ in range(M)]
    for idx, server in enumerate(servers):
        server.idx = idx + 1

    def client_generator():
        client_id = 0
        while True:
            yield env.timeout(random.expovariate(LAMBDA))
            client_id += 1
            chosen_server = min(servers, key=lambda s: len(s.queue))
            env.process(client(env, client_id, chosen_server))

    env.process(client_generator())
    env.run(until=SIM_TIME)

    avg_wait_time = total_wait_time / total_clients if total_clients else 0
    avg_clients_in_queue = total_wait_time / SIM_TIME
    avg_utilization = utilization / (M * SIM_TIME)

    print("\nResultados:")
    print(f"Tiempo promedio de un cliente en la cola: {avg_wait_time:.2f} minutos")
    print(f"Número de clientes en la cola en promedio: {avg_clients_in_queue:.2f}")
    print(f"Grado o factor de utilización de cada cajero: {avg_utilization:.2f}")


run_simulation()


0.05 - Cliente 1 llega al supermercado
0.05 - Cliente 1 comienza a ser atendido en la caja 1. Tiempo de espera: 0.00 minutos
0.34 - Cliente 1 deja la caja 1
0.42 - Cliente 2 llega al supermercado
0.42 - Cliente 2 comienza a ser atendido en la caja 1. Tiempo de espera: 0.00 minutos
0.49 - Cliente 3 llega al supermercado
0.55 - Cliente 4 llega al supermercado
0.55 - Cliente 4 comienza a ser atendido en la caja 2. Tiempo de espera: 0.00 minutos
0.83 - Cliente 2 deja la caja 1
0.83 - Cliente 3 comienza a ser atendido en la caja 1. Tiempo de espera: 0.34 minutos
0.84 - Cliente 4 deja la caja 2
0.98 - Cliente 5 llega al supermercado
1.11 - Cliente 3 deja la caja 1
1.11 - Cliente 5 comienza a ser atendido en la caja 1. Tiempo de espera: 0.13 minutos
1.18 - Cliente 6 llega al supermercado
1.22 - Cliente 5 deja la caja 1
1.22 - Cliente 6 comienza a ser atendido en la caja 1. Tiempo de espera: 0.05 minutos
1.24 - Cliente 6 deja la caja 1
1.31 - Cliente 7 llega al supermercado
1.31 - Cliente 7 co

### Explicacion de tiempos

Los tiempos son, en efecto, pequeños, pero esto está en consonancia con los parámetros definidos. La razón de estos tiempos cortos es que hay 3 cajas disponibles, y en la mayoría de los casos, cuando un cliente llega, hay una caja libre para atenderlo de inmediato.

### Explicacion codigo

Proceso de Poisson:
El proceso de Poisson es una forma de modelar eventos que ocurren de manera aleatoria a lo largo del tiempo, donde la probabilidad de que ocurra más de un evento en un intervalo de tiempo infinitesimalmente pequeño es prácticamente cero. En esta simulación, la llegada de clientes al supermercado sigue un proceso de Poisson. Esto significa que en cualquier momento, un cliente podría llegar, pero la probabilidad de que dos o más clientes lleguen al mismo instante es extremadamente baja.

En términos prácticos, usamos la distribución exponencial (que está relacionada con el proceso de Poisson) para determinar el tiempo entre llegadas sucesivas. Si se dice que hay λ clientes que llegan por hora en promedio (la tasa del proceso de Poisson), el tiempo entre las llegadas sigue una distribución exponencial con un parámetro λ.

En el código, esto se refleja en la línea:

**yield env.timeout(random.expovariate(rate))**

Donde rate es λ y random.expovariate(rate) genera tiempos aleatorios entre llegadas basados en la distribución exponencial.

Distribución Exponencial:
Además de modelar el tiempo entre llegadas, la distribución exponencial se usa para modelar el tiempo de servicio del cajero. Esto se basa en la suposición de que el tiempo que un cajero tarda en atender a un cliente es variable y sigue esta distribución.

Esto es modelado en el código con la línea:

**service_time = random.expovariate(LAMBDA)**
Esto significa que el tiempo que toma servir a un cliente es una variable aleatoria con una distribución exponencial, nuevamente con el parámetro λ.

Sistema de Cola:
El comportamiento de la cola (los clientes esperando ser atendidos) es el núcleo de esta simulación. En esta simulación, cuando un cliente llega, él o ella selecciona la caja con la fila más corta. Si hay múltiples cajas con la misma longitud de fila más corta, el cliente elige aleatoriamente una de esas cajas. Esto es modelado por:

**min(servers, key=lambda s: len(s.queue))**

Donde servers es la lista de cajas y se selecciona la caja con la fila más corta.